In [1]:

import pandas as pd
import pydicom
from PIL import Image
import matplotlib.pyplot as plt
from datetime import datetime
from torch.utils.data import DataLoader, ConcatDataset
from torch.utils.data import Dataset
import torch.optim as optim
from torchvision import transforms
import numpy as np
import torch
from torch import nn
import os
from codes.metrics import relative_root_mean_squared_error, r_score
from math import sqrt
from sklearn.metrics import r2_score, mean_squared_error
from torch.utils.tensorboard import SummaryWriter

2024-03-18 13:16:43.252105: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-18 13:16:43.253669: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-18 13:16:43.286069: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-18 13:16:44.007738: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Load the Excel file containing DICOM file paths and masses
training_root = 'New_Target_Trainng Data.xlsx'
test_root = 'New_Target_Test Data.xlsx'
root_folder = 'New_Training_Data_FINAL/'
root_folder_test = 'New_Test Data/'
df = pd.read_excel(training_root)

In [3]:
# Hyperparameters
# We can adjust the learning_rate here to improve the model performance, scheduler is used to reduce the learning rate at specific epochs
# The batch size can be adjusted based on the available memory
# If apply_augmentations is set to True, the model will apply augmentations to the images to improve the model performance
load_saved_model = False
apply_augmentations = True
training_epoch = 6000
batch_size = 16
learning_rate = 0.001
momentum = 0.9
weight_decay = 0.0005
scheduler = 1

device = "cuda:1"
logdir = "logs/"

In [4]:
# The Data class is used to load the DICOM files and masses from the Excel file


class DC_Data(Dataset):
    def __init__(self, root_csv, root='', augmentation=None, png=False):
        """
        Initializes the dataset object.
        
        Parameters:
        - root_csv: Path to the Excel file containing filenames and mass values.
        - root: Root directory where the DICOM files are located.
        - augmentation: Transformations to be applied to the images. If None, a default resize transformation is applied.
        - png: A boolean flag indicating whether the images are stored in PNG format instead of DICOM. This is primarily used to load the segmentation masks
        as the augmented data from unsupervised segmentation model.
        
        Note: The file codes/data.py contains the DC_Data class which is used to load the DICOM files and masses from the Excel file, however
        it extended with the depth and inverse depth prediction tasks.
        """
        self.df = pd.read_excel(root_csv)
        self.root = root
        self.png = png
        self.transform = transforms.Compose([transforms.Resize((256, 256))]) if augmentation is None \
                            else transforms.Compose([augmentation, transforms.Resize((256, 256))]) 
        if self.png:
            self.df['image_path'] = self.df.apply(lambda x: os.path.join(self.root, x[0].replace('.dcm', '.png')), axis=1)
            self.df = self.df[self.df['image_path'].apply(os.path.exists)]
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        if self.png:
            # Load the image directly as a PNG
            image_path = self.root + self.df.iloc[index, 0].replace('.dcm', '.png')
            image = Image.open(image_path).convert('L')
        else:
            dicom_path = self.root + self.df.iloc[index, 0]
            dicom_data = pydicom.dcmread(dicom_path)
            slope = dicom_data.RescaleSlope if "RescaleSlope" in dicom_data else 1
            intercept = dicom_data.RescaleIntercept if "RescaleIntercept" in dicom_data else 0
            image = dicom_data.pixel_array.astype(np.float32) * slope + intercept
            image = torch.from_numpy(image.transpose(0,1)/255)
            image = image.unsqueeze(0)
        
        image = self.transform(image)
        if not isinstance(image, torch.Tensor):
            image = transforms.functional.to_tensor(image)
        
        mass = self.df.iloc[index, 1]
        
        # The return is expected to be [1, 256, 256] and [1] for the image and mass respectively
        return image, mass

In [5]:
from codes.helpers import ScaleToCenterTransform, tensor_to_numpy

train_set = DC_Data(training_root, root=root_folder)

if apply_augmentations:
    # Geometry Augmentation, horizontal flip, grayscale, and zoom
    zoom_augmentation = ScaleToCenterTransform(output_size=256, scale_factor=1.5)
    augmented_set1 = DC_Data(training_root, root=root_folder, augmentation=transforms.RandomHorizontalFlip(p=0.5))
    augmented_set2 = DC_Data(training_root, root_folder, augmentation=transforms.RandomRotation(degrees=45))
    augmented_set3 = DC_Data(training_root, root_folder, augmentation=transforms.Grayscale(num_output_channels=1))
    augmented_set4 = DC_Data(training_root, root_folder, augmentation=zoom_augmentation)
    augmented_set5 = DC_Data(training_root, root_folder, augmentation=transforms.RandomAffine(degrees=45, translate=(0.1, 0.1), scale=(0.8, 1.2), shear=15))

    # ROI Augmention, we have to define the ROI using 4_Generate_GT_Segmentation.ipynb and manully save into the Training_Generated/Mask folder
    mask_folder = 'Training_Generated/Mask/'
    augmented_set6 = DC_Data(training_root, mask_folder, augmentation=transforms.ToTensor(), png=True)
    
    # Concat the original and augmented datasets
    train_set = ConcatDataset([train_set, augmented_set1, augmented_set2, augmented_set3, augmented_set4, augmented_set5, augmented_set6])

# DataLoader is used to load the data in batches
dataloader_train = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)

test_set = DC_Data(test_root, root=root_folder_test)
dataloader_test = DataLoader(test_set, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)

print(f'Train size: {len(train_set)}, Test size: {len(test_set)}')

Train size: 1563, Test size: 29


In [6]:
for i in range(4):
    image, mass = train_set[i] 
    print(image.shape, mass)

torch.Size([1, 256, 256]) 0.26152158010240906
torch.Size([1, 256, 256]) 0.25422173274596016
torch.Size([1, 256, 256]) 0.2544004400440044
torch.Size([1, 256, 256]) 0.28149076394690364


In [7]:
# The architecture code is defined in file architecture/fused_transformer.py
# The diagram is provided to understand the architecture
from architecture.fused_transformer import UNet

model = UNet(n_channels=1, n_classes=1).to(device)
optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
writer = SummaryWriter(log_dir=logdir)

In [8]:
# The code to make sure the model input and output are correct, we can test the first batch to fed into the network
data = next(iter(dataloader_train))

test_data = data[0]
output_test = model(test_data.to(device))
print(output_test.shape)

torch.Size([16, 1])


In [9]:
from tqdm import tqdm

class Engine(object):
    def __init__(self, model, optimizer, device, ema=None):
        # Initialize the Engine with the model, optimizer, and the device it's running on.
        self.model = model
        self.optimizer = optimizer
        self.device = device
        # Current epoch of training.
        self.cur_epoch = 0
        # Number of iterations the training has run.
        self.cur_iter = 0
        # The best validation epoch, used to track the epoch with the best validation performance.
        self.bestval_epoch = 0
        # Lists to track the training and validation losses.
        self.train_loss = []
        self.val_loss = []
        # Criterion for calculating loss. Here, it's Mean Squared Error Loss for regression tasks.
        self.criterion = torch.nn.MSELoss(reduction='mean')

    """ Block to begin training """
    def train(self, dataloader_train):
        loss_epoch = 0.
        num_batches = 0
        # Set the model to training mode.
        self.model.train()
        
        # Train loop
        # tqdm is used to display the training progress for each epoch.
        predictions = []
        ground_truths = []
        pbar = tqdm(dataloader_train, desc='Train Epoch {}'.format(self.cur_epoch))
        for data in pbar:
            # efficiently zero gradients
            # Zero the gradients before running the backward pass.
            self.optimizer.zero_grad(set_to_none=True)
            images = data[0].to(self.device, dtype=torch.float32)   # Image that will be fed into network
            gt_mass = data[1].to(self.device, dtype=torch.float32)  # Ensure gt_mass is a float tensor, The ground truth of mass

            # Pass the images through the model to get predictions.
            pred_mass = self.model(images)

            # The output should have the same dimensions as the target
            if pred_mass.ndim > gt_mass.ndim:
                pred_mass = pred_mass.squeeze()

            # Calculate the loss, backpropagation, and optimization
            loss = self.criterion(pred_mass, gt_mass)
            loss.backward()
            # Perform a single optimization step (parameter update).
            self.optimizer.step()

            # Aggregate the loss for the epoch
            loss_epoch += float(loss.item())
            num_batches += 1
            
            predictions.extend(pred_mass.detach().cpu().numpy().flatten())
            ground_truths.extend(gt_mass.detach().cpu().numpy().flatten())
            pbar.set_description("Loss: {:.4f}".format(loss.item()))
            
        pbar.close()
        avg_loss = loss_epoch / num_batches
        self.train_loss.append(avg_loss)
        
        r2 = r2_score(ground_truths, predictions)
        r = r_score(ground_truths, predictions)
        rmse = sqrt(mean_squared_error(ground_truths, predictions))
        
        writer.add_scalar("Training/Overall_Loss", avg_loss, self.cur_epoch)
        writer.add_scalar("Training/Overall_R", r, self.cur_epoch)
        writer.add_scalar("Training/Overall_R2", r2, self.cur_epoch)
        writer.add_scalar("Training/Overall_RMSE", rmse, self.cur_epoch)
        
        np_predictions = np.array(predictions)
        np_ground_truths = np.array(ground_truths)
        overall_rrmse = relative_root_mean_squared_error(np_predictions, np_ground_truths)
        writer.add_scalar("Training/Overall_Relative_RMSE", overall_rrmse, self.cur_epoch)

        self.cur_epoch += 1
        pbar.set_description("Epoch: {}, Average Loss: {:.4f}".format(self.cur_epoch, avg_loss))

    def test(self, dataloader_test):
        self.model.eval()  # Set the model to evaluation mode
        loss_epoch = 0.
        num_batches = 0
        
        # Prepare to collect predictions and ground truth
        predictions = []
        ground_truths = []
        
        with torch.no_grad():  # No need to calculate gradients
            pbar = tqdm(dataloader_test, desc='Test Epoch {}'.format(self.cur_epoch))
            for data in pbar:
                images = data[0].to(self.device, dtype=torch.float32)
                gt_mass = data[1].to(self.device, dtype=torch.float32)

                pred_mass = self.model(images)

                # The output should have the same dimensions as the target
                if pred_mass.ndim > gt_mass.ndim:
                    pred_mass = pred_mass.squeeze()

                loss = self.criterion(pred_mass, gt_mass)
                loss_epoch += float(loss.item())
                num_batches += 1
                
                # We want to put this back on the CPU to calculate the metrics
                predictions.extend(pred_mass.cpu().numpy().flatten())
                ground_truths.extend(gt_mass.cpu().numpy().flatten())
                pbar.set_description("Test Loss: {:.4f}".format(loss.item()))

        avg_loss = loss_epoch / num_batches
        self.val_loss.append(avg_loss)
        
        # Calculate the R2 score, R score, and RMSE for overall test set
        r2 = r2_score(ground_truths, predictions)
        r = r_score(ground_truths, predictions)
        rmse = sqrt(mean_squared_error(ground_truths, predictions))
        
        print(f"Test Epoch: {self.cur_epoch}, Average Loss: {avg_loss:.4f}")
        print(f"R2 Score: {r2:.4f}, R Score: {r:.4f}, RMSE: {rmse:.4f}")
        
        writer.add_scalar("Testing/Overall_Loss", avg_loss, self.cur_epoch)
        writer.add_scalar("Testing/Overall_R2_Score", r2, self.cur_epoch)
        writer.add_scalar("Testing/Overall_R_Score", r, self.cur_epoch)
        writer.add_scalar("Testing/Overall_RMSE", rmse, self.cur_epoch)
        
        np_predictions = np.array(predictions)
        np_ground_truths = np.array(ground_truths)
        overall_rrmse = relative_root_mean_squared_error(np_predictions, np_ground_truths)
        writer.add_scalar("Testing/Overall_Relative_RMSE", overall_rrmse, self.cur_epoch)
        
        print("{:<10} {:<15} {:<15} {:<15}".format('Sample', 'Predicted', 'GT', 'Relative RMSE'))
        for sample_num, (pred, gt) in enumerate(zip(predictions, ground_truths), 1):
            # Calculate the relative RMSE for each instances
            relative_rmse = relative_root_mean_squared_error(pred, gt)
            writer.add_scalar(f"Testing/Relative_RMSE/Sample_{sample_num}", relative_rmse, self.cur_epoch)
            writer.add_scalars(f'TestingPredictionvsGT/Sample_{sample_num}', {
                'Prediction': pred,
                'Ground Truth': gt,
            }, self.cur_epoch)
            print("{:<10} {:<15.5f} {:<15.5f} {:<15.5f}".format(sample_num, pred, gt, relative_rmse))

        return avg_loss


In [10]:
from codes.scheduler import CyclicCosineDecayLR

# The scheduler is used to reduce the learning rate at specific epochs
if scheduler:
	scheduler = CyclicCosineDecayLR(optimizer,
	                                init_decay_epochs=150,
	                                min_decay_lr=2.5e-6,
	                                restart_interval = 10,
	                                restart_lr=12.5e-5,
	                                warmup_epochs=20,
	                                warmup_start_lr=2.5e-6)


trainer = Engine(model, optimizer, device, ema=None)

# Load the saved model if load_saved_model is set to True
if load_saved_model:
	model.load_state_dict(torch.load('logs/final_model.pth'))
 
# Count the total number of trainable parameters
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print ('======Total trainable parameters: ', params)

for epoch in range(trainer.cur_epoch, training_epoch):
	trainer.train(dataloader_train)

	# Test the model every 20 epochs and save it to logs folder
	if (epoch + 1) % 20 == 0:
		trainer.test(dataloader_test)
		torch.save(model.state_dict(), os.path.join('logs', 'final_model.pth'))
	if scheduler:
		scheduler.step()


/home/cqilab/anaconda3/envs/sam/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


======Total trainable parameters:  149084442


Train Epoch 0:   0%|          | 0/98 [00:00<?, ?it/s]

Test Loss: 3926.4768: 100%|██████████| 2/2 [00:00<00:00,  4.92it/s]


Test Epoch: 1, Average Loss: 2222.4234
R2 Score: -0.6077, R Score: 0.5364, RMSE: 45.2343
Sample     Predicted       GT              Relative RMSE  
1          4.87615         5.04922         0.03428        
2          5.14825         14.22000        0.63796        
3          4.48482         0.07432         59.34201       
4          4.39954         0.14031         30.35621       
5          4.77615         2.56000         0.86568        
6          5.40714         75.56802        0.92845        
7          5.40222         32.32000        0.83285        
8          3.93913         0.98394         3.00343        
9          4.95987         7.36000         0.32610        
10         5.86028         53.32000        0.89009        
11         4.74894         3.57000         0.33023        
12         4.86315         3.85000         0.26316        
13         5.42863         13.57483        0.60010        
14         4.21102         0.22750         17.50974       
15         4.07787        

...

Test Epoch: 4, Average Loss: 395.5724
R2 Score: 0.7116, R Score: 0.9092, RMSE: 19.1585
Sample     Predicted       GT              Relative RMSE  
1          54.08938        84.90561        0.36295        
2          1.11138         0.14031         6.92099        
3          43.14389        32.32000        0.33490        
4          55.11437        60.76912        0.09305        
5          3.42165         3.57000         0.04155        
6          46.46347        53.32000        0.12859        
7          9.47542         8.50000         0.11476        
8          1.12111         0.22750         3.92791        
9          1.28512         2.56000         0.49800        
10         7.64084         16.74877        0.54380        
11         1.14046         0.07432         14.34463       
12         33.46968        22.78601        0.46887        
13         54.09190        64.84397        0.16581        
14         2.07529         5.04922         0.58899        
15         12.44258        1

Test Loss: 186.6201: 100%|██████████| 2/2 [00:00<00:00,  5.12it/s]


Test Epoch: 5, Average Loss: 434.5212
R2 Score: 0.6384, R Score: 0.9201, RMSE: 21.4515
Sample     Predicted       GT              Relative RMSE  
1          41.85197        53.32000        0.21508        
2          14.52910        11.76000        0.23547        
3          59.50224        128.67818       0.53759        
4          0.58860         0.98394         0.40179        
5          4.87476         5.04922         0.03455        
6          14.05226        8.50000         0.65321        
7          33.95779        84.90561        0.60005        
8          0.29855         0.07432         3.01688        
9          51.72692        100.05196       0.48300        
10         11.88532        7.36000         0.61485        
11         14.68917        13.57483        0.08209        
12         29.90397        32.32000        0.07475        
13         42.87827        66.37000        0.35395        
14         46.72126        60.76912        0.23117        
15         0.44803         0

Loss: 336.3978:  35%|███▍      | 34/98 [00:05<00:09,  6.77it/s]


KeyboardInterrupt: 

In [ ]:
torch.save(model.state_dict(), os.path.join('logs', 'final_model.pth'))

NameError: name 'writer' is not defined